In [ ]:
seed = 42
gamma = 0.99
mac-steps_per_episode = 10000
env = gym.make("CartPole-v0")
env.seed(seed)
eps = np.finfo(np.float32).eps.item()

In [ ]:
num_inputs = 4
num_actions = 2
num_hidden = 128

inputs = layers.Input(shape = (num_inputs,))
common = layers.Dense(num_hidden, activation = "relu")(inputs)
action = layers.DEnse(num_actions, activation = "softmax")(common)
critic = layers.Dense(1)(common)

model = keras.Model(inputs = inputs, output = [action, critic])

In [ ]:
while True:
  state = env.reset()
  episode_reward = 0
  with tf.GradientTape() as tape:
    for timestep in range(1, max_steps_per_episode):
      state = tf.convert_to_tensor(state)
      state = tf.expand_dims(state, 0)

      action_probs, critic_value = model(state)
      critic-value_history.append(critic_value[0, 0])

      action = np.random.choice(num_actions, p = np.squeeze(action_probs))
      action_probs_history.append(tf.math.log(action_probs[0, action]))

      # Apply the sampeled action in our environment
      state, reward, done, _ = env.step(action)
      rewards_history.append(reward)
      episode_reward += reward

      if done:
        break

      runnung_reward = 0,05 * episode_reward + (1 - 0.05) * running_reward

      returns = []
      discounted_sum = 0
      for r in rewards_history[::-1]:
        disconnected_sum = r + gamma * discounted_sum
        returns.insert(0, discounted_sum)

      returns = np.array(returns)
      returns = (returns - np.mean(returns)) / (np.std(returns) + eps)
      returns = returns.tolist()

      history = zip(action_probs_history, critic_value_history, returns)
      actor_losses = []
      critic_losses = []
      for log_prob, value, ret in history:
        diff = ret - value
        actor_losses.append(-log_prob * diff)

        critic_losses.append(
            huber_loss(tf.expand_dims(value, 0), tf.expand_dims(ret, 0))
        )

      loss_value = sum(actor_losses) + sum(critic_losses)
      grads = tape.gradient(loss_value, model.trainable_variables)
      optimizer.apply_gradients(zip(grads, model.trainable_variables))

      action_probs_history.clear()
      critic_value_history.clear()
      rewards_history.clear()

    episode_count += 1
    if episode_count % 10 == 0:
      template = "running reward: {:.2f} at episode {}"
      print(template.format(running_reward, episode_cout))

    if running_reward > 150:
      print("Solved at episode {}!".format(episode_count))
      break

--------------------------------------------------------------------------------

In [2]:
!pip install tesorflow == 2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2

ERROR: Invalid requirement: '=='
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 3.4 MB/s eta 0:00:00


In [15]:
!pip install pygame 

import os
os.environ['SDL_VIDEODRIVER']='dummy'
import pygame
pygame.display.set_mode((640,480))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<Surface(640x480x32 SW)>

In [16]:
import gym
import random

In [17]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [18]:
states

4

In [19]:
actions
# left or right

2

In [20]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    # env.reset(): Step을 실행하다가 epsiode가 끝나서 이를 초기화해서 재시작해야할 때, 초기 State를 반환
    # 새로운 에피소드(initial environment)를 불러온다.(reset)
    done = False
    score = 0
    
    while not done:
        env.render()
        # env.render(): Graphic User Interface (GUI)로 현재 진행상황을 출력하는 함수
        # 행동(action)을 취하기 이전에 환경에 대해 얻은 관찰값(observation)적용하여 그린다.
        action = random.choice([0, 1])
        # random.choice([0, 1]): 0과 1중 하나를 랜덤으로 뽑아준다.
        n_state, reward, done, info = env.step(action)
        # env.step(): 행동(action)을 취하기 이후에 환경에 대해 얻은 관찰값(observation)적용하여 제어
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:27.0
Episode:2 Score:15.0
Episode:3 Score:35.0
Episode:4 Score:11.0
Episode:5 Score:24.0
Episode:6 Score:24.0
Episode:7 Score:20.0
Episode:8 Score:18.0
Episode:9 Score:32.0
Episode:10 Score:18.0


In [21]:

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
# Dense: fully connected layer 모듈
# Dense(8, input_dim=4, init='uniform', activation='relu')) = (출력 뉴런수, 입력뉴런수, 가중치 초기화방법, 활성화 함수)
# Flatten: 추출된 주요 특징을 fully connected layer에 전달하기 위해 1차원 자료로 바꿔주는 layer
from tensorflow.keras.optimizers import Adam

In [22]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape = (1, states)))
    model.add(Dense(24, activation = 'relu'))
    # 출력 뉴런수 24개, 활성화 함수 'relu'
    model.add(Dense(24, activation = 'relu'))
    model.add(Dense(actions, activation = 'linear'))
    
    return model

In [23]:
model = build_model(states, actions)

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 4)                 0         
                                                                 
 dense (Dense)               (None, 24)                120       
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


In [25]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [26]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit = 10000, window_length = 1)
    dqn = DQNAgent(model = model, memory = memory, policy = policy, 
                   nb_actions = actions, nb_steps_warmup = 10, target_model_update = 1e-2)
    
    return dqn

# rl.agents.dqn.DQNAgent(model, policy=None, test_policy=None, enable_double_dqn=True, enable_dueling_network=False, dueling_type='avg')
# nb_steps(integer): Number of training steps to be performed.
# nb_steps_warmup: some schemes where the learning rate changes in a pre-determined way, to protect your oscillate parameters in the first time.
# target_model_update: 얼마나 자주 target model을 업데이트 할것인지(>1=업데이트를 많이, <1=업데이트를 적게)

In [27]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=10000, visualize=False, verbose=1)

# compile(self, optimizer, metrics=[])/ mae(mean absolute error)
# fit(self, env, nb_steps, action_repetition=1, callbacks=None, verbose=1, visualize=False, nb_max_start_steps=0, start_step_policy=None, log_interval=10000, nb_max_episode_steps=None)
# verbose: By setting verbose 0, 1 or 2 you just say how do you want to 'see' the training progress for each epoch.

AttributeError: ignored

In [ ]:
scores = dqn.test(env, nb_episodes = 10, visualize = False)
print(np.mean(scores.history['episode_reward']))

In [ ]:
scores = dqn.test(env, nb_episodes = 10, visualize = True)

In [ ]:
_ = dqn.test(env, nb_episodes = 10, visualize = True)

In [ ]:
dqn.save_weights('dqn_weights.h5f', overwrite = True)
# weight값들을 저장

In [ ]:
del model
del dqn
del env

In [ ]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [ ]:
dqn.load_weights('dqn_weights.h5f')

In [ ]:
_ = dqn.test(env, nb_episodes = 10, visualize = True)